In [2]:
import pandas as pd
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
dg_df = pd.read_csv('/content/gdrive/MyDrive/DG_out.csv')

In [21]:
print(dg_df)

                                                  tx_hash  indegree  \
0       0437cd7f8525ceed2324359c2d0ba26006d92d856a9c20...         0   
1       f4184fc596403b9d638783cf57adfe4c75c605f6356fbc...         1   
2       ea44e97271691990157559d0bdd9959e02790c34db6c00...         1   
3       a16f3ce4dd5deb92d98ef5cf8afeaf0775ebca408f708b...         1   
4       591e91f809d716912ca1d4a9295e70c3e78bab077683f7...         1   
...                                                   ...       ...   
707493  ae5fab5bb90868739d2b986c9bf5cc4103823821d41d92...         1   
707494  23da552489f3a94864ec9bde396a3e428f6b4ae83a4b50...         1   
707495  ea443fea9fe00797748f883f955fef1851deba4675ab1c...         1   
707496  8e4a656b4499139c20d3ac5d0cd6a9851f1f97cb02d7b6...         1   
707497  29a3d0877c555ba44fd51ffd69c0ace326662c51e15889...         1   

        outdegree      in_btc     out_btc   total_btc  mean_in_btc  \
0               1    0.000000   50.000000   50.000000     0.000000   
1      

In [24]:
dg_df.isna().sum()

,0
tx_hash,0
indegree,0
outdegree,0
in_btc,0
out_btc,0
total_btc,1
mean_in_btc,1
mean_out_btc,1
in_malicious,1
out_malicious,1


In [8]:
# get rid of last column in dg_df and don't use tx_hash in variables
df = dg_df.iloc[:-1, :]

In [9]:
df

,tx_hash,indegree,outdegree,in_btc,out_btc,total_btc,mean_in_btc,mean_out_btc,in_malicious,out_malicious,is_malicious,out_and_tx_malicious,all_malicious
0,0437cd7f8525ceed2324359c2d0ba26006d92d856a9c20...,0,1,0.000000,50.000000,50.000000,0.000000,50.000000,0.0,0.0,0.0,0.0,0.0
1,f4184fc596403b9d638783cf57adfe4c75c605f6356fbc...,1,2,50.000000,50.000000,100.000000,50.000000,25.000000,0.0,0.0,0.0,0.0,0.0
2,ea44e97271691990157559d0bdd9959e02790c34db6c00...,1,1,10.000000,10.000000,20.000000,10.000000,10.000000,0.0,0.0,0.0,0.0,0.0
3,a16f3ce4dd5deb92d98ef5cf8afeaf0775ebca408f708b...,1,1,40.000000,30.000000,70.000000,40.000000,30.000000,0.0,0.0,0.0,0.0,0.0
4,591e91f809d716912ca1d4a9295e70c3e78bab077683f7...,1,2,30.000000,30.000000,60.000000,30.000000,15.000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
707492,0eb5a6bac102143df2b32943750c5aad4c2a23eb162c84...,1,1,108.777088,108.387088,217.164175,108.777088,108.387088,0.0,0.0,0.0,0.0,0.0
707493,ae5fab5bb90868739d2b986c9bf5cc4103823821d41d92...,1,2,27.853730,27.853730,55.707459,27.853730,13.926865,0.0,0.0,0.0,0.0,0.0
707494,23da552489f3a94864ec9bde396a3e428f6b4ae83a4b50...,1,2,108.387088,108.387088,216.774175,108.387088,54.193544,0.0,0.0,0.0,0.0,0.0
707495,ea443fea9fe00797748f883f955fef1851deba4675ab1c...,1,1,26.463729,26.213729,52.677459,26.463729,26.213729,0.0,0.0,0.0,0.0,0.0


In [24]:
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
import random

In [25]:
random.seed(420)

In [96]:
df_majority = df[df["all_malicious"] == 0]
df_minority = df[df["all_malicious"] == 1]
# Downsample the majority class
df_majority_downsampled = df_majority.sample(n=3000, random_state=42)
df_balanced = pd.concat([df_majority_downsampled, df_minority])
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

In [97]:
selected_columns = ["indegree", "outdegree", "in_btc", "out_btc", "total_btc"]
X = df_balanced[selected_columns]  # replace with your actual target column
y = df_balanced["all_malicious"]

In [98]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.1,
    stratify=y,  # <-- This preserves the class distribution
    random_state=42
)

In [99]:
X_train = sm.add_constant(X_train)
X_test = sm.add_constant(X_test)

In [100]:
#find median and stdev of each numerical term
model = sm.Logit(y_train, X_train)

result = model.fit()

# Show summary
print(result.summary())

         Current function value: 0.245204
         Iterations: 35
                           Logit Regression Results                           
Dep. Variable:          all_malicious   No. Observations:                 3144
Model:                          Logit   Df Residuals:                     3138
Method:                           MLE   Df Model:                            5
Date:                Wed, 07 May 2025   Pseudo R-squ.:                  0.3986
Time:                        01:39:38   Log-Likelihood:                -770.92
converged:                      False   LL-Null:                       -1282.0
Covariance Type:            nonrobust   LLR p-value:                9.989e-219
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -4.0168      0.153    -26.186      0.000      -4.317      -3.716
indegree       0.0896      0.024      3.668      0.000       0.04

/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [101]:
#find
y_pred_proba = result.predict(X_test)
#make them understand what the variables are and the range they
y_pred = (y_pred_proba >= 0.5).astype(int)

In [102]:
# not matched correctly we need to title the variables from the github instead there is no column name for any of the
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification report:\n", classification_report(y_test, y_pred))

Accuracy: 0.8714285714285714
Confusion matrix:
 [[260  41]
 [  4  45]]
Classification report:
               precision    recall  f1-score   support

         0.0       0.98      0.86      0.92       301
         1.0       0.52      0.92      0.67        49

    accuracy                           0.87       350
   macro avg       0.75      0.89      0.79       350
weighted avg       0.92      0.87      0.88       350

